In [105]:
#import pandas as pd
import json
import numpy as np
from tqdm import tqdm_notebook as tqdm
import uuid
import weaviate 
from weaviate.batch import Batch 
import helper
import weaviate

# initiate the Weaviate client
client = weaviate.Client("http://localhost:8081")
client.timeout_config = (3, 200)


# load all review data 
reviewData = []
with open('data/sample_Home_and_Kitchen_5.json', 'r') as f:
    for l in tqdm(f):
        reviewData.append(json.loads(l))

In [108]:
# load all metadata
count = 0
data = []
with open('data/sample_meta_Home_and_Kitchen.json', 'r') as f:
    for l in tqdm(f):
        data.append(json.loads(l))
        count = count + 1
        if count >= 3000:
          break


productData = []

for i in range(0, 3000):
    hasprice="price" in data[i]
    hastitle="title" in data[i]
    hasdescription="description" in data[i]
    hasmaincat='main_cat' in data[i]
    #print(i,  (hasprice and hastitle and hasdescription))
    if (hasprice and hastitle and hasdescription and hasmaincat)==True:
        if len(data[i]['title'])<200:
            productData.append(data[i])
    else:
        continue


for i in range(0, len(productData)): 
   productData[i]['description']=" ".join(productData[i]['description'])
   cleaned_text = productData[i]['description'].replace("\\", "")
   productData[i]['description']=cleaned_text
   productData[i]['price']=productData[i]['price'].replace("$","").replace(",","")
   productData[i]['price']=float(productData[i]['price'])




/var/folders/4c/x3415rcx4pb9drxwpfyzk32h0000gn/T/ipykernel_72623/3562440711.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for l in tqdm(f):


0it [00:00, ?it/s]

In [ ]:
## here needs a step to clean and process / filter out data 
## len(productData['title']) <100 

In [112]:
client.schema.delete_all() # delete all classes

In [113]:
import weaviate

# skipped product "brand", it can be it's own class, for simplicity, skip it for now 
# skipped product "category" list of strings (text?) not sure how to type it yet, or if category should be it's own class?
# skipped "similar_item"
schema = {
    "classes": [
        {
            # name of the class
            "class": "Product",
            # class properties
            "properties": [
                {
                    "name": "asin",
                    "dataType": ["string"]
                },
                {
                    "name": "title",
                    "dataType": ["text"]
                },
                {
                    "dataType": ["number"],
                    "description": "The price product in dollars",
                    "name": "price"
                },
                {
                    "dataType": ["text"],
                    "name": "productDescription",
                    "description": "description of product"

                },
                {
                    "dataType": ["string"],
                    "name": "mainCat",
                    "description": "main category of the product in amazon "
                }
            ]
        }
    ]
}
client.schema.create(schema)

In [114]:
def prettify(json_dict): 
    print(json.dumps(json_dict, indent=2))

prettify(client.schema.get())

{
  "classes": [
    {
      "class": "Product",
      "invertedIndexConfig": {
        "cleanupIntervalSeconds": 60
      },
      "moduleConfig": {
        "text2vec-transformers": {
          "poolingStrategy": "masked_mean",
          "vectorizeClassName": true
        }
      },
      "properties": [
        {
          "dataType": [
            "string"
          ],
          "moduleConfig": {
            "text2vec-transformers": {
              "skip": false,
              "vectorizePropertyName": false
            }
          },
          "name": "asin"
        },
        {
          "dataType": [
            "text"
          ],
          "moduleConfig": {
            "text2vec-transformers": {
              "skip": false,
              "vectorizePropertyName": false
            }
          },
          "name": "title"
        },
        {
          "dataType": [
            "number"
          ],
          "description": "The price product in dollars",
          "moduleConfig":

In [115]:
# test that the helper uuid generating function works 
helper.generate_uuid('product', productData[350]['title']+productData[350]['asin'])

'28c5e357-2d19-55d0-a685-1756ad0a7e47'

In [116]:
from weaviate.batch import Batch # for the typing purposes

# so far i only got to successfully import 'asin', 'title'

def add_product(batch: Batch, product_data: dict) -> str:
    product_object = {
        'asin': product_data['asin'],
        'title': product_data['title'],
        'main_cat': product_data['main_cat'],
        'description': product_data['description'],             
        #'feature': product_data['feature'] if 'feature' in product_data else '',
        'price': product_data['price']
    }
    # generate an UUID for the Author
    product_id = helper.generate_uuid('product', product_data['title']+product_data['asin'])
   
    # add article to the batch
    batch.add_data_object(  
        data_object=product_object,
        class_name='Product',
        uuid=product_id
    ) 
    return product_id





In [118]:
from tqdm import trange

# just to test a small number to see if it works 

for i in trange(500, 1000):
    product_id = add_product(client.batch, productData[i])    
    if i % 20 == 0:
        # submit the objects from the batch to weaviate
        client.batch.create_objects()
status_objects = client.batch.create_objects()
client.batch.flush()

100%|██████████| 500/500 [02:09<00:00,  3.86it/s]


In [ ]:
result = client.query.get(class_name='Product', properties="title")\
    .do()
print(f"Number of reviews returned: {len(result['data']['Get']['Product'])}")
#result

Number of reviews returned: 100
